In [181]:
import pandas as pd
import os

## Описание данных
InvoiceNo — номер транзакции  
StockCode — код товара  
Description — описание товара  
Quantity — количество единиц товара, добавленных в заказ  
InvoiceDate — дата транзакции  
UnitPrice — цена за единицу товара  
CustomerID — id клиента  
Country — страна, где проживает клиент  

In [182]:
os.listdir(os.getcwd())

['les_5_hw_1.ipynb', '.ipynb_checkpoints', '5_data.csv.zip']

In [183]:
retail = pd.read_csv('5_data.csv.zip', compression = 'zip', encoding='iso-8859-1')

In [184]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [185]:
retail.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


In [186]:
retail_columns = retail.columns

In [187]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [188]:
retail.shape[0]

541909

In [189]:
retail_dd = retail.drop_duplicates()

In [190]:
retail.shape[0] - retail_dd.shape[0]

5268

In [191]:
# Методом берем строку и по первой букве определяем начинаетя она с С или нет
# На основе условия создаем датафрейм
canseled_invoce = retail_dd[retail_dd['InvoiceNo'].str.startswith('C')]
canseled_invoce['InvoiceNo'].count()

9251

In [192]:
retail_dd = retail_dd.query('Quantity > 0')

In [193]:
retail_dd.groupby('Country', as_index=False).agg({'CustomerID' : 'count'}).query('Country == "Germany"')

,Country,CustomerID
14,Germany,9027


In [194]:
customer_germany = retail_dd.query('Country == "Germany"').groupby('CustomerID', as_index=False).agg({'InvoiceNo': 'nunique'})
customer_germany

,CustomerID,InvoiceNo
0,12426.0,1
1,12427.0,3
2,12468.0,2
3,12471.0,30
4,12472.0,7
...,...,...
89,13814.0,2
90,13815.0,4
91,13816.0,2
92,13817.0,2


In [195]:
invoice_qu80 = customer_germany['InvoiceNo'].quantile(q = 0.80)
invoice_qu80

7.0

In [196]:
customer_germany_qu80 = customer_germany.query('InvoiceNo > @invoice_qu80')

In [197]:
germany_top = customer_germany_qu80['CustomerID'].to_list()
germany_top

[12471.0,
 12474.0,
 12476.0,
 12481.0,
 12500.0,
 12524.0,
 12569.0,
 12600.0,
 12619.0,
 12621.0,
 12626.0,
 12647.0,
 12662.0,
 12705.0,
 12708.0,
 12709.0,
 12712.0,
 12720.0]

In [198]:
top_retail_germany = retail_dd.query('CustomerID in @germany_top')

In [199]:
top_retail_germany.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4175 entries, 1109 to 541730
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    4175 non-null   object 
 1   StockCode    4175 non-null   object 
 2   Description  4175 non-null   object 
 3   Quantity     4175 non-null   int64  
 4   InvoiceDate  4175 non-null   object 
 5   UnitPrice    4175 non-null   float64
 6   CustomerID   4175 non-null   float64
 7   Country      4175 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 293.6+ KB


In [200]:
top_retail_germany.query('StockCode != "POST"')\
                  .groupby('StockCode')\
                  .agg({'Quantity' : 'count'})\
                  .sort_values('Quantity', ascending = False)\
                  .reset_index()\
                  .head(5)

,StockCode,Quantity
0,22326,52
1,22328,38
2,22423,34
3,20719,30
4,22961,28


In [201]:
retail_dd['Revenue'] =retail_dd['UnitPrice'] * retail_dd['Quantity']

In [202]:
retail_dd.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [209]:
top_invoice = retail_dd.groupby('InvoiceNo').agg({'Revenue' : 'sum'}).sort_values('Revenue', ascending=False).head(5).reset_index()

In [210]:
top_invoice

,InvoiceNo,Revenue
0,581483,168469.60
1,541431,77183.60
2,574941,52940.94
3,576365,50653.91
4,556444,38970.00


In [211]:
top_invoice.InvoiceNo.tolist()

['581483', '541431', '574941', '576365', '556444']